In [1]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from scipy import ndimage
from PIL import Image
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn import svm
from sklearn import preprocessing as prp
from helpers_img_my import *
from Post_processing import *
#from helpers_img import *
from sklearn import model_selection as modsel
from preprocessing_my import *

In [2]:
def post_processing(label,threshold,size_min,verbarg,horbarg,size_image):
    label = complete_lines(label,threshold)
    label = remove_isolated_connected_component(label,size_min)
    label = clean_garbage_vert(label,verbarg,size_image)
    label = clean_garbage_hor(label,horbarg,size_image)
    label = remove_isolated_connected_component(label,size_min)
    #label = complete_semilines(label,0.85, size_image)
    return label

In [ ]:
# Settings
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
patch_size = 16 # each patch is 16*16 pixels
deg=2

root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"
files = os.listdir(image_dir)

n=int(len(files)/2)
k_fold=2 #number of folds in the k-fold cross-validation

ims = [load_image(image_dir + files[i]) for i in range(n)]
gt_ims = [load_image(gt_dir + files[i]) for i in range(n)]

seed = 1
np.random.seed(seed)

kf=modsel.KFold(n_splits=k_fold,shuffle=True)
kf.get_n_splits(ims)
splits=kf.split(ims)
lambdas=np.logspace(0,8,15) # 1/lambda

mean_f1=np.zeros(lambdas.size)

trains=[]
tests=[]
for (train_id,test_id) in splits:
    trains.append(train_id)
    tests.append(test_id)
    
print(trains)
print(tests)

ims=[add_features(ims[i]) for i in range(len(ims))]
ims,gt_ims = rotation(ims,gt_ims)
ims,gt_ims = flip(ims,gt_ims)
print(len(ims))

ims = [img_crop(ims[i], patch_size, patch_size) for i in range(len(ims))]
gt_ims = [img_crop(gt_ims[i], patch_size, patch_size) for i in range(len(gt_ims))]

ims = np.asarray([ims[i][j] 
                    for i in range(len(ims)) 
                    for j in range(len(ims[i]))])

gt_ims = np.asarray([gt_ims[i][j] 
                    for i in range(len(gt_ims)) 
                    for j in range(len(gt_ims[i]))])

ims=[extract_features(ims[i]) for i in range(len(ims))]
ims = np.asarray([poly_features(ims[i],deg) for i in range(len(ims))])
gt_ims = np.asarray([value_to_class(np.mean(gt_ims[i])) for i in range(len(gt_ims))])

for id_lam,lam in enumerate(lambdas):
    print('lambda: '+str(id_lam))
    nb_f1_te = np.zeros(k_fold)
    for train,test in zip(trains,tests):
        print('here')
        im_tr = np.zeros((len(train)*625*8,325))
        gt_tr = np.zeros((len(train)*625*8))
        inter = 0
        for idx in train:
            temps = [ims[625*idx + (625*n)*k: 625*(idx+1) + (625*n)*k] for k in np.arange(0,8)]
            gt_temps = [gt_ims[625*idx + (625*n)*k :625*(idx+1) + (625*n)*k] for k in np.arange(0,8)]
            for (temp,gt_temp) in zip(temps,gt_temps):
                im_tr[625*inter:625*(inter+1),:]= temp
                gt_tr[625*inter:625*(inter+1)]=gt_temp
                inter = inter + 1
            
        im_te = np.zeros((len(train)*625,325))
        gt_te = np.zeros((len(train)*625))
        inter = 0
        for idx in test:
            im_te[625*inter:625*(inter+1),:] = ims[625*idx : 625*(idx+1)]
            gt_te[625*inter:625*(inter+1)] = gt_ims[625*idx : 625*(idx+1)]
            inter = inter + 1
        
        logreg = linear_model.LogisticRegression(C=lam, class_weight="balanced")
        logreg.fit(im_tr, gt_tr)

        Z_te = logreg.predict(im_te)

        Z_pp=[]
        for i in range(len(gt_te)):
            Z_pp = Z_pp + post_processing(Z_te[i*625:(i+1)*625],18,9,3,3,25)
            
        nb_f1_te[ind]=compute_F1(gt_te, Z_pp)
        print(nb_f1_te[ind])
        
    mean_f1[id_lam]=nb_f1_te.mean()
    

best_lambda=lambdas[np.argmax(mean_f1)]

[array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 18, 20, 25,
       28, 30, 33, 34, 37, 43, 44, 47]), array([ 2,  3, 13, 17, 19, 21, 22, 23, 24, 26, 27, 29, 31, 32, 35, 36, 38,
       39, 40, 41, 42, 45, 46, 48, 49])]
[array([ 2,  3, 13, 17, 19, 21, 22, 23, 24, 26, 27, 29, 31, 32, 35, 36, 38,
       39, 40, 41, 42, 45, 46, 48, 49]), array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 18, 20, 25,
       28, 30, 33, 34, 37, 43, 44, 47])]


In [ ]:
a=np.arange(1,8)
print(a)